# Lib def

In [3]:
import numpy as np
from sympy.simplify.fu import fu, L, TR9, TR10i, TR11
from sympy import factor, sin, cos, powsimp, exp
from sympy import re, im, I, E, Abs, S, conjugate
from sympy import symbols, Function, lambdify, simplify, preorder_traversal, Float, latex, pprint
from IPython.display import display
from math import pi

def tensor_product(a, b):
    if ("__len__" not in dir(a)):
        return a*b
    elif ("__len__" not in dir(b)):
        return b*a
    else:
        res = [tensor_product(a[i//len(b)], b[i%len(b)]) for i in range(len(a)*len(b))]
        return np.array(res)

def transform_vect(M, v):
    if (len(M) == len(v) and len(M[0]) == len(v)):
        return matmul(M, v)
    else:
        return v

def matmul(a, b):
    return np.matmul(a, b)

In [4]:
from sympy import init_printing

init_printing(use_latex='mathjax')

In [92]:
def diag_ones(dim):
    return np.array([[int(j == i) for j in range(dim)] for i in range(dim)], dtype='object')

def id(n):
    return diag_ones(2**n)

def rz(theta):
    return np.array([[exp(-I*theta/2), 0], [0, exp(I*theta/2)]])

def rx(theta):
    return np.array([[cos(theta/2), -I*sin(theta/2)], [-I*sin(theta/2), cos(theta/2)]])

def ry(theta):
    return np.array([[cos(theta/2), -sin(theta/2)], [sin(theta/2), cos(theta/2)]])

def p(theta):
    return np.array([[1, 0], [0, exp(I*theta/2)]])
    
def x():
    return np.array([[0, 1], [1, 0]])

def y():
    return np.array([[0, -1j], [1j, 0]])

def z():
    return np.array([[1, 0], [0, -1]])

def cnot(direction=1):
    return cgate(x(), direction)

def cgate(M, direction=1):
    base = diag_ones(2*len(M))

    if direction == 1:
        k = len(M)
    else:
        k = 0

    for i in range(len(M)):
        for j in range(len(M[0])):
            base[i+k][j+k] = M[i][j]

    return np.array(base)

def cgate2(M, c, t, n=None):
    if n is None:
        n = max(c, t)+1

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])

    proj0x0 = np.outer(ket_0, dagger(ket_0))
    proj1x1 = np.outer(ket_1, dagger(ket_1))

    proj0s = [id(1) for _ in range(n)]
    proj0s[c] = proj0x0
    proj0s[t] = id(1)

    proj1s = [id(1) for _ in range(n)]
    proj1s[c] = proj1x1
    proj1s[t] = M

    return np.array(mtp2(proj0s)+mtp2(proj1s))

def dagger(M):
    return M.transpose()

def conjugate(M):
    res = [[i for i in row] for row in M]

    for i in range(len(M)):
        for j in range(len(M)):
            if type(res[i][j]) == type(1+1j):
                res[i][j] = np.conjugate(res[i][j])
            if type(res[i][j] == e):
                res[i][j] = e(-res[i][j].args())
            elif type(-res[i][j] == e):
                res[i][j] = -e((-res[i][j]).args())

    return res

def idx_to_vect(bit_str):
    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])

    kets = [ket_0 if b == "0" else ket_1 for b in bit_str]

    return multitensor_product(kets)

def braket(sv1, sv2):
    sv1_dagger = dagger(sv1)

    res = matmul(sv1_dagger, sv2)

    return res

def multitensor_product(mats):
    res = None

    for mat in mats:
        if res is None:
            res = mat
        else:
            res = tensor_product(res, mat)

    return np.array(res)

def mtp2(mats):
    res = None
    tempmat = None

    for mat in mats[::-1]:
        if res is None:
            if tempmat is None:
                tempmat = mat
            else:
                res = tensor_product(mat, tempmat)
        else:
            res = tensor_product(mat, res)

    return res

def idx_to_vect2(bit_str):
    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])

    kets = [ket_0 if b == "0" else ket_1 for b in bit_str]

    return mtp2(kets)

def tfv(M, v):
    return transform_vect(M, v)

ket_0 = np.array([1, 0])
ket_1 = np.array([0, 1])
H = 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])

<!-- ![Sample SimpleSA circuit](../data/misc/sample-simplesa-circuit.png) -->
<img src="../data/misc/sample-simplesa-circuit.png" alt="Sample SimpleSA circuit" width=300px align=left>

# Circuit Test

In [37]:
x0 = symbols('x_0', real=True)
x1 = symbols('x_1', real=True)
x2 = symbols('x_2', real=True)
pi = symbols('\\pi', real=True)
# pi = 3.141592653589

In [38]:
psi0 = idx_to_vect("00")
psi1 = tfv(mtp2([rx(pi/2), rx(pi/2)]), psi0)
psi2 = tfv(cgate2(x(), 0, 1), psi1)
psi3 = tfv(mtp2([id(1), p(2*(pi-x0)*(pi-x1))]), psi2)
psi4 = tfv(cgate2(x(), 0, 1), psi3)
psi5 = tfv(mtp2([rx(-pi/2), rx(-pi/2)]), psi4)

psi = psi5

In [39]:
# derive expression for <00|psi>

raw00 = braket(idx_to_vect("00"), psi).expand(trig=True).expand()

braket00 = simplify(raw00, maxn=17, chop=True)
temp = braket00
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket00 = braket00.subs(a, round(a, 15))

display(braket00)

⎛                  ⎛   2        ⎞                                           ⎞ 
⎜                ⅈ⋅⎝\pi  + x₀⋅x₁⎠                                           ⎟ 
⎜(1 - cos(\pi))⋅ℯ                   ⎛   4⎛\pi⎞      4⎛\pi⎞⎞  ⅈ⋅\pi⋅(x₀ + x₁)⎟ 
⎜──────────────────────────────── + ⎜sin ⎜───⎟ + cos ⎜───⎟⎟⋅ℯ               ⎟⋅
⎝               4                   ⎝    ⎝ 4 ⎠       ⎝ 4 ⎠⎠                 ⎠ 

                 
                 
 -ⅈ⋅\pi⋅(x₀ + x₁)
ℯ                
                 

In [40]:
# calculate P(|00>) = |<00|psi>|

p00 = simplify(re(braket00)*re(braket00)+im(braket00)*im(braket00))

display(p00)

                                                                              
                                                       ⎛                      
                                                       ⎜               ⎛   2  
              2    2⎛   2                          ⎞   ⎜-cos(\pi) - cos⎝\pi  -
(1 - cos(\pi)) ⋅sin ⎝\pi  - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠   ⎝                      
──────────────────────────────────────────────────── + ───────────────────────
                         16                                                   

                                                                              
                               ⎛     2                                ⎞      ⎛
                        ⎞   cos⎝- \pi  + \pi⋅x₀ + \pi⋅x₁ + \pi - x₀⋅x₁⎠   cos⎝
 \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠ + ─────────────────────────────────────────── + ────
                                                 2                            
───────────────────────────────────────────────────

In [41]:
# derive expression for <01|psi>

raw01 = braket(idx_to_vect("01"), psi).expand(trig=True).expand()

braket01 = simplify(raw01, maxn=17, chop=True)
temp = braket01
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket01 = braket01.subs(a, round(a, 15))

display(braket01)

  ⎛       ⎛   2        ⎞                ⎛   2        ⎞                        
  ⎜     ⅈ⋅⎝\pi  + x₀⋅x₁⎠    2⎛\pi⎞    ⅈ⋅⎝\pi  + x₀⋅x₁⎠    ⅈ⋅\pi⋅(x₀ + x₁)    ⎛
ⅈ⋅⎜- 2⋅ℯ                ⋅cos ⎜───⎟ + ℯ                 + ℯ               ⋅cos⎜
  ⎝                          ⎝ 4 ⎠                                           ⎝

    ⎞                                    
\pi⎞⎟  -ⅈ⋅\pi⋅(x₀ + x₁)    ⎛\pi⎞    ⎛\pi⎞
───⎟⎟⋅ℯ                ⋅sin⎜───⎟⋅cos⎜───⎟
 2 ⎠⎠                      ⎝ 4 ⎠    ⎝ 4 ⎠

In [42]:
# calculate P(|01>) = |<01|psi>|

p01 = simplify(re(braket01)*re(braket01)+im(braket01)*im(braket01))

display(p01)

⎛                                                        2                    
⎜               ⎛   ⎛   2                          ⎞    ⎞                     
⎜(cos(\pi) + 1)⋅⎝cos⎝\pi  - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠ - 1⎠       2⎛   2        
⎜───────────────────────────────────────────────────────── + sin ⎝\pi  - \pi⋅x
⎝                            2                                                

                             ⎞                    
                             ⎟                    
                  ⎞    2⎛\pi⎞⎟    2⎛\pi⎞    2⎛\pi⎞
₀ - \pi⋅x₁ + x₀⋅x₁⎠⋅cos ⎜───⎟⎟⋅sin ⎜───⎟⋅cos ⎜───⎟
                        ⎝ 2 ⎠⎠     ⎝ 4 ⎠     ⎝ 4 ⎠

In [43]:
p0 = simplify(p00+p01)

display(p0)

⎛                                                        2                    
⎜               ⎛   ⎛   2                          ⎞    ⎞                     
⎜(cos(\pi) + 1)⋅⎝cos⎝\pi  - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠ - 1⎠       2⎛   2        
⎜───────────────────────────────────────────────────────── + sin ⎝\pi  - \pi⋅x
⎝                            2                                                

                             ⎞                                                
                             ⎟                                     2    2⎛   2
                  ⎞    2⎛\pi⎞⎟    2⎛\pi⎞    2⎛\pi⎞   (cos(\pi) - 1) ⋅sin ⎝\pi 
₀ - \pi⋅x₁ + x₀⋅x₁⎠⋅cos ⎜───⎟⎟⋅sin ⎜───⎟⋅cos ⎜───⎟ + ─────────────────────────
                        ⎝ 2 ⎠⎠     ⎝ 4 ⎠     ⎝ 4 ⎠                            

                                                                              
                          ⎞   ⎛                  ⎛   2                        
 - \pi⋅x₀ - \pi⋅x₁ + x₀⋅x₁⎠   ⎝2⋅cos(\pi) + 2⋅cos⎝

In [47]:
pi_val = 3.141592653589
expression = sin(pi_val**2-pi*x0-pi*x1+x0*x1)**2
res = simplify(expression)

display(res)

   2                                            
sin (\pi⋅x₀ + \pi⋅x₁ - x₀⋅x₁ - 9.86960440108437)

# SVAO

In [45]:
t0 = symbols('\\theta_0', real=True)
t1 = symbols('\\theta_1', real=True)
t2 = symbols('\\theta_2', real=True)
t3 = symbols('\\theta_3', real=True)
t4 = symbols('\\theta_4', real=True)
pi = symbols('\\pi', real=True)

In [46]:
# |psi0> = |0000>
psi0 = idx_to_vect("0000")

# |psi1> = Rx(t0)*Rx(t1)*I*Rx(t2) |psi0>
psi1 = tfv(mtp2([rx(t0), rx(t1), id(1), rx(t2)]), psi0)

# |psi2> = CNOT[0, 3] |psi1>
psi2 = tfv(cgate2(x(), 0, 3), psi1)

# |psi3> = CNOT[3, 1] |psi2>
psi3 = tfv(cgate2(x(), 3, 1), psi2)

# |psi4> = CNOT[1, 0]*I^2 |psi3>
psi4 = tfv(mtp2([cgate2(x(), 1, 0), id(1), id(1)]), psi3)

# |psi5> = Rx(t0)*Rx(t1)*I*Rx(t2) |psi4>
psi5 = tfv(mtp2([rx(t0), rx(t1), id(1), rx(t2)]), psi4)

# |psi6> = CNOT[0, 1]*I^2 |psi5>
psi6 = tfv(mtp2([cgate2(x(), 0, 1), id(1), id(1)]), psi5)

# |psi7> = CNOT[1, 3] |psi6>
psi7 = tfv(cgate2(x(), 1, 3), psi6)

# |psi8> = CNOT[3, 0] |psi7>
psi8 = tfv(cgate2(x(), 3, 0), psi7)

# |psi9> = CNOT[2, 3] |psi8>
psi9 = tfv(cgate2(x(), 2, 3), psi8)

# |psi10> = CNOT[3, 2] |psi9>
psi10 = tfv(cgate2(x(), 3, 2), psi9)

# |psi11> = I^2*Rx(t4)*Rx(t3) |psi10>
psi11 = tfv(mtp2([id(1), id(1), rx(t4), rx(t3)]), psi10)

# |psi12> = CNOT[2, 3] |psi11>
psi12 = tfv(cgate2(x(), 2, 3), psi11)

# |psi13> = CNOT[3, 2] |psi12>
psi13 = tfv(cgate2(x(), 3, 2), psi12)

# |psi14> = I^2*Rx(t4)*Rx(t3) |psi13>
psi14 = tfv(mtp2([id(1), id(1), rx(t4), rx(t3)]), psi13)

In [47]:
# for e in psi14:
#     print("---")
#     display(e)

In [48]:
# # derive expression for <0000|psi10>

# raw0000 = braket(idx_to_vect("0000"), psi14).expand(trig=True).expand()

# braket0000 = simplify(raw0000, maxn=17, chop=True)
# temp = braket0000
# for a in preorder_traversal(temp):
#     if isinstance(a, Float):
#         braket0000 = braket0000.subs(a, round(a, 15))

# display(braket0000)

In [49]:
# # calculate P(|0000>) = |<0000|psi10>|

# p0000 = simplify(re(braket0000)*re(braket0000)+im(braket0000)*im(braket0000))

# display(p0000)

In [114]:
p0000 = ((-sin(t1)-sin(t2)-sin(t4)-sin(t0+t3)+sin(t1-t2)-sin(t1+t4)-sin(t2+t4)-sin(-t0+t1+t3)+sin(-t0+t3+t4)-sin(t0-t2+t3)-sin(-t1+t2+t4)+sin(-t0+t2+t3+t4)+sin(t0-t1+t3+t4)+sin(t0+t1+t2-t3)+sin(t0+t1+t2+t3+t4))**2)/256+((+cos(t1)+cos(t2)+cos(t4)+cos(t0+t3)+cos(t1-t2)+cos(t1+t4)+cos(t2+t4)+cos(-t0+t1+t3)+cos(-t0+t3+t4)+cos(t0-t2+t3)+cos(-t1+t2+t4)+cos(-t0+t2+t3+t4)+cos(t0-t1+t3+t4)+cos(t0+t1+t2-t3)+cos(t0+t1+t2+t3+t4)+1)**2)/256

display(p0000)

                                                                              
(-sin(\theta₁) - sin(\theta₂) - sin(\theta₄) - sin(\theta₀ + \theta₃) + sin(\t
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
heta₁ - \theta₂) - sin(\theta₁ + \theta₄) - sin(\theta₂ + \theta₄) - sin(-\the
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
ta₀ + \theta₁ + \theta₃) + sin(-\theta₀ + \theta₃ + \theta₄) - sin(\theta₀ - \
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                 

In [51]:
# # derive expression for <0100|psi10>

# raw0100 = braket(idx_to_vect("0100"), psi14).expand(trig=True).expand()

# braket0100 = simplify(raw0100, maxn=17, chop=True)
# temp = braket0100
# for a in preorder_traversal(temp):
#     if isinstance(a, Float):
#         braket0100 = braket0100.subs(a, round(a, 15))

# display(braket0100)

In [52]:
# # calculate P(|0100>) = |<0100|psi10>|

# p0100 = simplify(re(braket0100)*re(braket0100)+im(braket0100)*im(braket0100))

# display(p0100)

In [53]:
p0100 = ((+sin(t1)-sin(t2)-sin(t4)+sin(t0+t3)-sin(t1-t2)+sin(t1+t4)-sin(t2+t4)-sin(-t0+t1+t3)-sin(-t0+t3+t4)+sin(t0-t2+t3)+sin(-t1+t2+t4)-sin(-t0+t2+t3+t4)+sin(t0-t1+t3+t4)+sin(t0+t1+t2-t3)+sin(t0+t1+t2+t3+t4))**2)/256+((-cos(t1)+cos(t2)+cos(t4)-cos(t0+t3)-cos(t1-t2)-cos(t1+t4)+cos(t2+t4)+cos(-t0+t1+t3)-cos(-t0+t3+t4)-cos(t0-t2+t3)-cos(-t1+t2+t4)-cos(-t0+t2+t3+t4)+cos(t0-t1+t3+t4)+cos(t0+t1+t2-t3)+cos(t0+t1+t2+t3+t4)+1)**2)/256

display(p0100)

                                                                              
(sin(\theta₁) - sin(\theta₂) - sin(\theta₄) + sin(\theta₀ + \theta₃) - sin(\th
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
eta₁ - \theta₂) + sin(\theta₁ + \theta₄) - sin(\theta₂ + \theta₄) - sin(-\thet
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                                              
a₀ + \theta₁ + \theta₃) - sin(-\theta₀ + \theta₃ + \theta₄) + sin(\theta₀ - \t
──────────────────────────────────────────────────────────────────────────────
                                                                              

                                                 

In [54]:
p0_denom = simplify(p0000+p0100)

display(p0_denom)

cos(\theta₂)   cos(\theta₄)   cos(2⋅\theta₁ - \theta₂)   cos(2⋅\theta₁ + \thet
──────────── + ──────────── + ──────────────────────── + ─────────────────────
     16             16                   32                         32        

a₂)   cos(\theta₂ - \theta₄)   cos(\theta₂ + \theta₄)   cos(2⋅\theta₃ + \theta
─── + ────────────────────── + ────────────────────── + ──────────────────────
                64                       64                        16         

₄)   cos(-\theta₀ + \theta₁ + \theta₃)   cos(\theta₀ - \theta₁ + \theta₃)   co
── + ───────────────────────────────── + ──────────────────────────────── + ──
                     64                                 64                    

s(-2⋅\theta₁ + \theta₂ + \theta₄)   cos(2⋅\theta₁ - \theta₂ + \theta₄)   cos(-
───────────────────────────────── + ────────────────────────────────── + ─────
               64                                   64                        

\theta₂ + 2⋅\theta₃ + \theta₄)   cos(\theta₂ + 2

In [55]:
# calculate P(O=|0>) = P(|0000>)/( P(|0000>)+P(|0100>) )

p0 = simplify(p0000/p0_denom)

display(p0)

                                                                              
                                                                              
                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
4⋅cos(\theta₂) + 4⋅cos(\theta₄) + 2⋅cos(2⋅\theta₁ - \theta₂) + 2⋅cos(2⋅\theta₁

                                                                              
                                                                              
                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
 + \theta₂) + cos(\theta₂ - \theta₄) + cos(\theta₂ + \theta₄) + 4⋅cos(2⋅\theta

                                                  

In [56]:
# p0_00 = (4*cos(2*t1+t2)+4*cos(2*t1-t2)+2*cos(2*t3+t4)-4*cos(t0+t1-2*t2-t3-2*t4)-4*cos(t0+t1-t2-t3-2*t4)-4*cos(t0-t1-t2-t3-2*t4)-4*cos(t0-t1-t2-t3-t4)-4*cos(t0-t1-t3)-4*cos(t0-t1-t3-2*t4)-4*cos(t0-t1-t3-t4)+4*cos(t2)-4*cos(t2-2*t3-t4)+8*cos(t4)+4*cos(2*t1-t2+t4)+4*cos(2*t1+t2+2*t3+t4)+4*cos(t0-t1+t2+t3+2*t4)+4*cos(t0-t1+t2+t3+t4)+4*cos(t0-t1+t3+2*t4)+4*cos(t0-t1+t3+t4)+4*cos(t0+t1-2*t2+t3)+4*cos(t0+t1-t2+t3)+4*cos(t0+t1+2*t2-t3)+4*cos(t0+t1+2*t2-t3+t4)+4*cos(t0+t1+2*t2+t3+2*t4)+4*cos(t0+t1+2*t2+t3+t4)+4*cos(t0+t1+t2-t3)+4*cos(t0+t1+t2-t3+t4)+4*cos(t0+t1+t2+t3+2*t4)+4*cos(t0+t1+t2+t3+t4)+4*cos(t2+2*t3+t4)+4*cos(t2+t4))/256

# display(p0_00)

In [57]:
# p0 = simplify(p0000/p0_00)

# display(p0)

In [58]:
cook_0 = 1.204406074425311*3.141592653589793*2
cook_1 = 0.4647590541304515*3.141592653589793*2
cook_2 = 0.17828484324013236*3.141592653589793*2
unappet_0 = 0.9555832350160824*3.141592653589793*2
unappet_1 = 1.208010784363145*3.141592653589793*2

In [59]:
p0.subs([(t0, cook_0), (t1, cook_1), (t2, cook_2), (t3, unappet_0), (t4, unappet_1)])

0.502193262935713

In [166]:
num = -2*cos(2*t1+t2)-2*cos(2*t1+t2-2*t3-t4)-2*cos(2*t1-t2)-2*cos(2*t3+t4)-4*cos(t0+t1-2*t2-t3-2*t4)-4*cos(t0+t1-t2-t3-2*t4)-4*cos(t0-t1-t2-t3-2*t4)-4*cos(t0-t1-t2-t3-t4)-4*cos(t0-t1-t3)-4*cos(t0-t1-t3-2*t4)-4*cos(t0-t1-t3-t4)+4*cos(t2)-4*cos(t2-2*t3-t4)+8*cos(t4)+4*cos(2*t1-t2+t4)+4*cos(2*t1+t2+2*t3+t4)+4*cos(t0-t1+t2+t3+2*t4)+4*cos(t0-t1+t2+t3+t4)+4*cos(t0-t1+t3+2*t4)+4*cos(t0-t1+t3+t4)+16

(num).subs([(t0, cook_0), (t1, cook_1), (t2, cook_2), (t3, unappet_0), (t4, unappet_1)])

24.8340089932729

# SXA

In [93]:
t0 = symbols('\\theta_0', real=True)
t1 = symbols('\\theta_1', real=True)
pi = symbols('\\pi', real=True)

In [94]:
# |phi0> = |00>
phi0 = idx_to_vect("00")

# |phi1> = Rx(t0)*Rx(t1) |phi0>
phi1 = tfv(mtp2([rx(t0), rx(t1)]), phi0)

# |phi2> = CNOT[0, 1] |phi1>
phi2 = tfv(cgate2(x(), 0, 1), phi1)

# |phi3> = CNOT[1, 0] |phi2>
phi3 = tfv(cgate2(x(), 1, 0), phi2)

# |phi4> = Rx(t0)*Rx(t1) |phi3>
phi4 = tfv(mtp2([rx(t0), rx(t1)]), phi3)

# |phi5> = CNOT[0, 1] |phi4>
phi5 = tfv(cgate2(x(), 0, 1), phi4)

# |phi6> = CNOT[1, 0] |phi5>
phi6 = tfv(cgate2(x(), 1, 0), phi5)

In [95]:
# derive expression for <00|phi6>

raw00 = braket(idx_to_vect("00"), phi6).expand(trig=True).expand()

braket00 = simplify(raw00, maxn=17, chop=True)
temp = braket00
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket00 = braket00.subs(a, round(a, 15))

display(braket00)

 ⅈ⋅\theta₀    ⅈ⋅\theta₁    ⅈ⋅(-\theta₀ - \theta₁)    
ℯ            ℯ            ℯ                         1
────────── + ────────── + ─────────────────────── + ─
    4            4                   4              4

In [96]:
# calculate P(|00>) = |<00|phi6>|

p00 = simplify(re(braket00)*re(braket00)+im(braket00)*im(braket00))

display(p00)

                                                      2                       
(sin(\theta₀) + sin(\theta₁) - sin(\theta₀ + \theta₁))    (cos(\theta₀) + cos(
─────────────────────────────────────────────────────── + ────────────────────
                           16                                                 

                                      2
\theta₁) + cos(\theta₀ + \theta₁) + 1) 
───────────────────────────────────────
         16                            

In [97]:
# derive expression for <01|psi10>

raw01 = braket(idx_to_vect("01"), phi6).expand(trig=True).expand()

braket01 = simplify(raw01, maxn=17, chop=True)
temp = braket01
for a in preorder_traversal(temp):
    if isinstance(a, Float):
        braket01 = braket01.subs(a, round(a, 15))

display(braket01)

   ⅈ⋅\theta₀    ⅈ⋅\theta₁        -ⅈ⋅(\theta₀ + \theta₁)
  ℯ            ℯ            1   ℯ                      
- ────────── + ────────── - ─ + ───────────────────────
      4            4        4              4           

In [98]:
# calculate P(|01>) = |<01|phi6>|

p01 = simplify(re(braket01)*re(braket01)+im(braket01)*im(braket01))

display(p01)

                                                      2                       
(sin(\theta₀) - sin(\theta₁) + sin(\theta₀ + \theta₁))    (-cos(\theta₀) + cos
─────────────────────────────────────────────────────── + ────────────────────
                           16                                                 

                                       2
(\theta₁) + cos(\theta₀ + \theta₁) - 1) 
────────────────────────────────────────
         16                             

In [99]:
p0_ = simplify(p00+p01)

display(p0_)

cos(\theta₀)   cos(\theta₀ + 2⋅\theta₁)   1
──────────── + ──────────────────────── + ─
     4                    4               2

In [100]:
# calculate P(O=|0>) = P(|00>)/( P(|00>)+P(|01>) )

f0 = simplify(p00/p0_)

display(f0)

                                                      2                       
(sin(\theta₀) + sin(\theta₁) - sin(\theta₀ + \theta₁))  + (cos(\theta₀) + cos(
──────────────────────────────────────────────────────────────────────────────
                                   4⋅(cos(\theta₀) + cos(\theta₀ + 2⋅\theta₁) 

                                      2
\theta₁) + cos(\theta₀ + \theta₁) + 1) 
───────────────────────────────────────
+ 2)                                   

In [109]:
slow_0 = 0.7436994864265065*3.141592653589796*2
slow_1 = 0.4019002667598174*3.141592653589796*2

In [110]:
f0.subs([(t0, slow_0), (t1, slow_1)])

0.508117328542717

In [123]:
num = 2*(cos(t1)+cos(t0+t1)+cos(t0-t1)+cos(2*t0+t1))

display(num)

2⋅cos(\theta₁) + 2⋅cos(\theta₀ - \theta₁) + 2⋅cos(\theta₀ + \theta₁) + 2⋅cos(2
⋅\theta₀ + \theta₁)

In [124]:
denom = (4*(cos(t0)+cos(t0+2*t1)+2))

display(denom)

4⋅cos(\theta₀) + 4⋅cos(\theta₀ + 2⋅\theta₁) + 8

In [125]:
num.subs([(t0, slow_0), (t1, slow_1)])

0.0326196492652886

In [126]:
denom.subs([(t0, slow_0), (t1, slow_1)])

4.01852026730583

In [138]:
(cos(t0)).subs([(t0, slow_0), (t1, slow_1)])

-0.0395769552259158

In [143]:
zero = 2*(cos(t1)+cos(t0+t1)+cos(t0-t1)+cos(2*t0+t1))/(4*(cos(t0)+cos(t0+2*t1)+2))

zero.subs([(t0, slow_0), (t1, slow_1)])

0.00811732854271707

In [155]:
(cos(t0)).subs([(t0, 0), (t1, slow_1)])

1

In [156]:
(cos(t1)).subs([(t0,0), (t1, slow_1)])

-0.815977158898529

In [157]:
(cos(t0+t1)).subs([(t0, 0), (t1, slow_1)])

-0.815977158898529